In [9]:
import argparse
import numpy as np
from data_loader import load_data

np.random.seed(555)

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='movie', help='which dataset to use')
parser.add_argument('--dim', type=int, default=16, help='dimension of entity and relation embeddings')
parser.add_argument('--n_hop', type=int, default=1, help='maximum hops')
parser.add_argument('--kge_weight', type=float, default=0.01, help='weight of the KGE term')
parser.add_argument('--l2_weight', type=float, default=1e-7, help='weight of the l2 regularization term')
parser.add_argument('--lr', type=float, default=0.02, help='learning rate')
parser.add_argument('--batch_size', type=int, default=1024, help='batch size')
parser.add_argument('--n_epoch', type=int, default=100, help='the number of epochs')
parser.add_argument('--n_memory', type=int, default=40, help='size of ripple set for each hop')
parser.add_argument('--item_update_mode', type=str, default='plus_transform',
                    help='how to update item at the end of each hop')
parser.add_argument('--using_all_hops', type=bool, default=True,
                    help='whether using outputs of all hops or just the last hop when making prediction')



parser.add_argument('--use_cuda', type=bool, default=True, help='whether to use gpu')
args = parser.parse_args(args=[])



In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score


class RippleNet(nn.Module):
    def __init__(self, args, n_entity, n_relation):
        super(RippleNet, self).__init__()

        self._parse_args(args, n_entity, n_relation)

        # 做Embedding 
        self.entity_emb = nn.Embedding(self.n_entity, self.dim) #n_e*dim # 从Embedding里面取数据是不会改变维度的
        self.relation_emb = nn.Embedding(self.n_relation, self.dim * self.dim) # n_r*dim*dim
        self.transform_matrix = nn.Linear(self.dim, self.dim, bias=False)  # dim*dim
        self.criterion = nn.BCELoss()

    def _parse_args(self, args, n_entity, n_relation):
        self.n_entity = n_entity
        self.n_relation = n_relation
        self.dim = args.dim
        self.n_hop = args.n_hop
        self.kge_weight = args.kge_weight
        self.l2_weight = args.l2_weight
        self.lr = args.lr
        self.n_memory = args.n_memory
        self.item_update_mode = args.item_update_mode
        self.using_all_hops = args.using_all_hops

    def forward(
        self,
        items: torch.LongTensor,  # 要预测的物品索引 注意要存在于KG里面
        labels: torch.LongTensor,
        memories_h: list,     
        memories_r: list,
        memories_t: list,
    ):
        # [batch size, dim]
        item_embeddings = self.entity_emb(items)
        h_emb_list = []
        r_emb_list = []
        t_emb_list = []
        # 从训练集中读取数据，每批次1024个
        for i in range(self.n_hop):
            # [batch size, n_memory, dim]
            h_emb_list.append(self.entity_emb(memories_h[i]))    
            # [batch size, n_memory, dim, dim]
            r_emb_list.append(
                self.relation_emb(memories_r[i]).view(
                    -1, self.n_memory, self.dim, self.dim      
                )
            )
            # [batch size, n_memory, dim]
            t_emb_list.append(self.entity_emb(memories_t[i]))

        o_list, item_embeddings = self._key_addressing(
            h_emb_list, r_emb_list, t_emb_list, item_embeddings
        )
        scores = self.predict(item_embeddings, o_list)

        return_dict = self._compute_loss(
            scores, labels, h_emb_list, t_emb_list, r_emb_list
        )
        return_dict["scores"] = scores

        return return_dict

    def _compute_loss(self, scores, labels, h_emb_list, t_emb_list, r_emb_list):
        base_loss = self.criterion(scores, labels.float())

        kge_loss = 0
        for hop in range(self.n_hop):
            # [batch size, n_memory, 1, dim]
            h_expanded = torch.unsqueeze(h_emb_list[hop], dim=2)
            # [batch size, n_memory, dim, 1]
            t_expanded = torch.unsqueeze(t_emb_list[hop], dim=3)
            # [batch size, n_memory, dim, dim]
            hRt = torch.squeeze(
                torch.matmul(torch.matmul(h_expanded, r_emb_list[hop]), t_expanded)
            )
            kge_loss += torch.sigmoid(hRt).mean()
        kge_loss = -self.kge_weight * kge_loss

        l2_loss = 0
        for hop in range(self.n_hop):
            l2_loss += (h_emb_list[hop] * h_emb_list[hop]).sum()
            l2_loss += (t_emb_list[hop] * t_emb_list[hop]).sum()
            l2_loss += (r_emb_list[hop] * r_emb_list[hop]).sum()
        l2_loss = self.l2_weight * l2_loss

        loss = base_loss + kge_loss + l2_loss
        return dict(base_loss=base_loss, kge_loss=kge_loss, l2_loss=l2_loss, loss=loss)

    # h_emb_list [n_hop, batch size, n_memory, dim]
    # r_emb_list [n_hop, batch size, n_memory, dim, dim]
    # t_emb_list [n_hop, batch size, n_memory, dim]
    def _key_addressing(self, h_emb_list, r_emb_list, t_emb_list, item_embeddings):
        o_list = []
        for hop in range(self.n_hop):
            # [batch_size, n_memory, dim, 1]
            h_expanded = torch.unsqueeze(h_emb_list[hop], dim=3)
            
            # [batch_size, n_memory, dim] 32*16
            Rh = torch.squeeze(torch.matmul(r_emb_list[hop], h_expanded))
            
            # [batch_size, dim, 1]  16*1
            v = torch.unsqueeze(item_embeddings, dim=2)
            
            # [batch_size, n_memory]
            probs = torch.squeeze(torch.matmul(Rh, v))
            
            # [batch_size, n_memory]
            probs_normalized = F.softmax(probs, dim=1)
            
            # [batch_size, n_memory, 1]
            probs_expanded = torch.unsqueeze(probs_normalized, dim=2)
            
            # [batch_size, dim]
            o = (t_emb_list[hop] * probs_expanded).sum(dim=1)
            
            item_embeddings = self._update_item_embedding(item_embeddings, o)
            o_list.append(o)
        return o_list, item_embeddings

    def _update_item_embedding(self, item_embeddings, o):
        if self.item_update_mode == "replace":
            item_embeddings = o
        elif self.item_update_mode == "plus":
            item_embeddings = item_embeddings + o
        elif self.item_update_mode == "replace_transform":
            item_embeddings = self.transform_matrix(o)
        elif self.item_update_mode == "plus_transform":
            item_embeddings = self.transform_matrix(item_embeddings + o)
        else:
            raise Exception("Unknown item updating mode: " + self.item_update_mode)
        return item_embeddings

    def predict(self, item_embeddings, o_list):
        y = o_list[-1]
        if self.using_all_hops:
            for i in range(self.n_hop - 1):
                y += o_list[i]

        # [batch_size]
        scores = (item_embeddings * y).sum(dim=1)
        return torch.sigmoid(scores)

    def evaluate(self, items, labels, memories_h, memories_r, memories_t):
        return_dict = self.forward(items, labels, memories_h, memories_r, memories_t)
        scores = return_dict["scores"].detach().cpu().numpy()
        labels = labels.cpu().numpy()
        auc = roc_auc_score(y_true=labels, y_score=scores)
        predictions = [1 if i >= 0.5 else 0 for i in scores]
        acc = np.mean(np.equal(predictions, labels))
        return auc, acc


In [11]:
import numpy as np
import torch



def train(args, data_info, show_loss):
    train_data = data_info[0]
    eval_data = data_info[1]
    test_data = data_info[2]
    n_entity = data_info[3]
    n_relation = data_info[4]
    ripple_set = data_info[5]

    model = RippleNet(args, n_entity, n_relation)
    if args.use_cuda:
        model.cuda()
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        args.lr,
    )

    for step in range(args.n_epoch):
        # training
        np.random.shuffle(train_data)
        start = 0
        while start < train_data.shape[0]:
            return_dict = model(*get_feed_dict(args, model, train_data, ripple_set, start, start + args.batch_size))
            loss = return_dict["loss"]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            start += args.batch_size
            if show_loss:
                print('%.1f%% %.4f' % (start / train_data.shape[0] * 100, loss.item()))

        # evaluation
        train_auc, train_acc = evaluation(args, model, train_data, ripple_set, args.batch_size)
        eval_auc, eval_acc = evaluation(args, model, eval_data, ripple_set, args.batch_size)
        test_auc, test_acc = evaluation(args, model, test_data, ripple_set, args.batch_size)

        print('epoch %d    train auc: %.4f  acc: %.4f    eval auc: %.4f  acc: %.4f    test auc: %.4f  acc: %.4f'
                % (step, train_auc, train_acc, eval_auc, eval_acc, test_auc, test_acc))
    # 保存模型
    torch.save(model.state_dict(), '../models/ripple100.pth') 
    net_args = []
    net_args.append(n_entity)
    net_args.append(n_relation)
    np.save('../models/net_args.npy', net_args)


# data: [[user, item, rate]...]
def get_feed_dict(args, model, data, ripple_set, start, end):
    items = torch.LongTensor(data[start:end, 1])  # 一维 [item1, item2...]
    labels = torch.LongTensor(data[start:end, 2])  # 一维
    memories_h, memories_r, memories_t = [], [], []
    # memories_h:
    # [[h1, h2,...],  第1层的所有头节点
    #  [...]...]      第2层..
    for i in range(args.n_hop):
        memories_h.append(torch.LongTensor([ripple_set[user][i][0] for user in data[start:end, 0]]))
        memories_r.append(torch.LongTensor([ripple_set[user][i][1] for user in data[start:end, 0]]))
        memories_t.append(torch.LongTensor([ripple_set[user][i][2] for user in data[start:end, 0]]))
    if args.use_cuda:
        items = items.cuda()
        labels = labels.cuda()
        memories_h = list(map(lambda x: x.cuda(), memories_h))
        memories_r = list(map(lambda x: x.cuda(), memories_r))
        memories_t = list(map(lambda x: x.cuda(), memories_t))
    return items, labels, memories_h, memories_r,memories_t


def evaluation(args, model, data, ripple_set, batch_size):
    start = 0
    auc_list = []
    acc_list = []
    model.eval()
    while start < data.shape[0]:
        auc, acc = model.evaluate(*get_feed_dict(args, model, data, ripple_set, start, start + batch_size))
        auc_list.append(auc)
        acc_list.append(acc)
        start += batch_size
    model.train()
    return float(np.mean(auc_list)), float(np.mean(acc_list))


In [12]:
show_loss = False
data_info = load_data(args)  # train_data, eval_data, test_data, n_entity, n_relation, ripple_set
ripple_set = data_info[5]
np.save('../models/ripple_set.npy', ripple_set)
np.save('../models/train_data.npy', data_info[0])
train(args, data_info, show_loss)


reading rating file ...
splitting dataset ...
reading KG file ...
constructing knowledge graph ...
constructing ripple set ...
epoch 0    train auc: 0.5261  acc: 0.5248    eval auc: 0.4822  acc: 0.4864    test auc: 0.5087  acc: 0.5029
epoch 1    train auc: 0.5650  acc: 0.5477    eval auc: 0.4927  acc: 0.5020    test auc: 0.5167  acc: 0.5260
epoch 2    train auc: 0.5867  acc: 0.5715    eval auc: 0.4958  acc: 0.5148    test auc: 0.5238  acc: 0.5167
epoch 3    train auc: 0.5896  acc: 0.5658    eval auc: 0.5041  acc: 0.5203    test auc: 0.5386  acc: 0.5277
epoch 4    train auc: 0.6031  acc: 0.5678    eval auc: 0.5058  acc: 0.5100    test auc: 0.5381  acc: 0.5241
epoch 5    train auc: 0.6082  acc: 0.5767    eval auc: 0.5043  acc: 0.5083    test auc: 0.5263  acc: 0.5049
epoch 6    train auc: 0.6075  acc: 0.5823    eval auc: 0.5014  acc: 0.4980    test auc: 0.5169  acc: 0.4987
epoch 7    train auc: 0.6132  acc: 0.5901    eval auc: 0.5013  acc: 0.5006    test auc: 0.5097  acc: 0.4904
epoch 8  

# Predict

In [ ]:
net_args = np.load('../models/net_args.npy')
ripple_set = np.load('../models/ripple_set.npy', allow_pickle=True
).item()

model = RippleNet(args, net_args[0], net_args[1])
if args.use_cuda:
    model.cuda()
model.load_state_dict(torch.load('../models/ripple100.pth')) # 加载模型的权重
model.eval() # 将模型设置为评估模式

RippleNet(
  (entity_emb): Embedding(1324, 16)
  (relation_emb): Embedding(1, 256)
  (transform_matrix): Linear(in_features=16, out_features=16, bias=False)
  (criterion): BCELoss()
)

In [ ]:
import pandas as pd
# 读取已保存的item_index.txt文件，并构建item_index_dict字典

def load_item_index(file_path):
    item_index_df = pd.read_csv(file_path, sep="\t")
    item_index_dict = dict(zip(item_index_df["item"], item_index_df["index"]))
    return item_index_dict, item_index_df

# 定义文件路径
saved_item_index_path = "../../process/item_index.txt"

# 尝试加载已保存的item_index.txt文件
try:
    item_index_dict, item_index_df = load_item_index(saved_item_index_path)
    print("Item index loaded successfully.")
except FileNotFoundError:
    print("Item index file not found. You may need to create it.")

# 然后可以使用get_item_or_index函数
def get_item_or_index(x):
    # 如果输入的是item名，就返回索引值
    if isinstance(x, str):
        return item_index_dict.get(x, None)
    # 如果输入的是索引值，就返回item名
    elif isinstance(x, int):
        return item_index_df.loc[item_index_df["index"] == x, "item"].values[0]
    # 否则，返回None
    else:
        return None


Item index loaded successfully.


In [ ]:
print(get_item_or_index('前端开发'))

97


In [ ]:
get_item_or_index('产品测试')

955

In [ ]:
ripple_set

defaultdict(list,
            {20171684310268: [([417,
                68,
                280,
                349,
                595,
                708,
                773,
                68,
                595,
                417,
                708,
                109,
                708,
                816,
                109,
                108,
                890,
                39,
                842,
                76,
                708,
                68,
                76,
                349,
                109,
                68,
                842,
                890,
                773,
                708,
                816,
                39,
                280,
                225,
                39,
                842,
                842,
                108,
                816,
                280],
               [0,
                0,
                0,
                0,
                0,
                0,
                0,
 

In [ ]:
'''
    self,
    items: torch.LongTensor,
    labels: torch.LongTensor,
    memories_h: list,
    memories_r: list,
    memories_t: list,
'''

memories_h, memories_r, memories_t = [], [], []
# 路线
route0 = get_item_or_index('前端开发')
route1 = get_item_or_index('后端开发')
route2 = get_item_or_index('移动开发')
route3 = get_item_or_index('网络安全')
route4 = get_item_or_index('硬件开发')
route5 = get_item_or_index('游戏开发')
route6 = get_item_or_index('产品运维')
route7 = get_item_or_index('产品测试')
route8 = get_item_or_index('人工智能')
route9 = get_item_or_index('算法设计')
route10 = get_item_or_index('云计算')
route11 = get_item_or_index('计算机视觉')
route12 = get_item_or_index('数据管理')
idx2route = {0: '前端开发', 1:'后端开发',2:'移动开发',3:'网络安全', 4:'硬件开发',5:'游戏开发',6:'产品运维',
             7:'产品测试',8:'人工智能',9:'算法设计',10:'云计算', 11:'计算机视觉', 12: '数据管理'}
items = torch.LongTensor([route0, route1, route2, route3,
                          route4, route5, route6,
                          route7, route8, route9,
                          route10, route11, route12])  # 需要至少两个不然报错
labels = torch.LongTensor(np.zeros(len(items)))            # 要至少两个不然报错

# 给第一个用户测试 一共十三个路线 
user = np.full(13, 20171684310268)# 要至少两个不然报错


for i in range(args.n_hop):
        memories_h.append(torch.LongTensor([ripple_set[user][i][0] for user in user]))
        memories_r.append(torch.LongTensor([ripple_set[user][i][1] for user in user]))
        memories_t.append(torch.LongTensor([ripple_set[user][i][2] for user in user]))
if args.use_cuda:
    items = items.cuda()
    labels = labels.cuda()
    memories_h = list(map(lambda x: x.cuda(), memories_h))
    memories_r = list(map(lambda x: x.cuda(), memories_r))
    memories_t = list(map(lambda x: x.cuda(), memories_t))


# model(*get_feed_dict(args, model, test_data, ripple_set, args.batch_size))
return_dict = model(items, labels, memories_h, memories_r, memories_t)
scores = return_dict['scores'].cpu().detach().numpy()



In [ ]:
import numpy as np

scores
# 对数组进行降序排序，并返回排序后的索引
idx = np.argsort(scores) [::-1]
# 输出排序后的概率和学习路线
for i in idx:
    print (f'概率：{scores [i]}, 路线：{idx2route[i]}')


概率：0.5000026822090149, 路线：数据管理
概率：0.5, 路线：计算机视觉
概率：0.5, 路线：云计算
概率：0.5, 路线：算法设计
概率：0.5, 路线：人工智能
概率：0.5, 路线：游戏开发
概率：0.5, 路线：网络安全
概率：0.5, 路线：后端开发
概率：0.5, 路线：前端开发
概率：0.4999934434890747, 路线：产品运维
概率：0.49999332427978516, 路线：产品测试
概率：0.4999873638153076, 路线：移动开发
概率：0.4999781847000122, 路线：硬件开发
